# Import NHP surfaces from NHP-Freesurfer into Pycortex
This pipeline assumes that you have already:    
- Have [Freesurfer](https://surfer.nmr.mgh.harvard.edu/) installed.
- Processed the NHP data with [NHP-Freesurfer](https://github.com/VisionandCognition/NHP-Freesurfer).     
- Installed pycortex as explained in the [README](https://github.com/VisionandCognition/NHP-pycortex/blob/master/README.md) file of this repo.

If this is the case you can follow the steps below to:     
- Import the NHP-Freesurfer subject into Pycortex (surfaces)   
- Import flatmaps and transform information between templates and surfaces
- Check if it worked using dummy data    

Finally, some instructions are provided to project *any* volumetric to surfaces with Pycortex.

## Import FS subject
This works for FS subjects that have been properly processed with [NHP-Freesurfer](https://github.com/VisionandCognition/NHP-Freesurfer). Some methods in Pycortex have been adapted to make them compatible with NHP pipelines. For example, `cortex.freesurfer.import_subjnhp` is an nhp adaptation of pycortex's `cortex.freesurfer.import_subj`. The original method is also still present, but `cortex.freesurfer.import_subjnhp` avoids trying to import data-files that do not exist for NHP-Freesurfer (like the subcortical segementation `aseg.mgz`).

In [24]:
import cortex
import os, shutil, subprocess
import numpy as np

In [25]:
subject='NMT' # should be the FS Subject name

In [26]:
# freesurfer subject folder
fsp = os.path.join(os.environ['SUBJECTS_DIR'],subject) # if FS is installed, this env_variable should give us the subjects directory
# manual-masks path with reference volumes
manmask_path = '/Users/chris/Dropbox/GIT_Support/NHP-BIDS/manual-masks' # NB! this is system dependent
#mm_func = os.path.join(manmask_path, 'sub-' + subject.lower(), 'func')

In [27]:
# import Freesurfer subject into pycortex
cortex.freesurfer.import_subjnhp(subject)

b'created by chris on Sat Jun 20 21:56:51 2020\n'
b'created by chris on Sat Jun 20 21:56:52 2020\n'
b'created by chris on Sat Jun 20 22:04:11 2020\n'
b'created by chris on Sat Jun 20 22:04:12 2020\n'
b'created by chris on Sat Jun 20 21:56:51 2020\n'
b'created by chris on Sat Jun 20 22:04:11 2020\n'
b'created by chris on Sat Jun 20 21:56:52 2020\n'
b'created by chris on Sat Jun 20 22:04:12 2020\n'
b'created by chris on Sat Jun 20 21:57:02 2020\n'
b'created by chris on Sat Jun 20 22:04:22 2020\n'


## Import FS subject
These flatmaps should have been generated with [NHP-Freesurfer](https://github.com/VisionandCognition/NHP-Freesurfer). Here we only import the full hemispheric flatmap, but you can do the same thing for other flatmaps (e.g., occipital). Make sure they are in the expected naming scheme though and create copiess (like we do below) if they're not.

In [28]:
# copy FS flatmaps to expected naming scheme (may not be necessary but is if you followed NHP-Freesurfer)
for xh in ['lh','rh']:
    src_file = os.path.join(fsp,'surf', xh + '.full.patch.flat')
    trg_file = os.path.join(fsp,'surf', xh + '.full.flat.patch.3d')
    shutil.copyfile(src_file,trg_file)

In [29]:
# also import FS flatmaps into pycortex
cortex.freesurfer.import_flat(subject,"full")

flatmap vertex locations for this subject, and will result
in deletion of the overlays.svg file and all cached info
for this subject (because flatmaps will fundamentally change).
Proceed? [y]/n:  


b'created by chris on Sat Jun 20 21:56:51 2020\n'
saving to /home/chris/Dropbox/CURRENT_PROJECTS/NHP_MRI/pycortex/db/NMT/surfaces/flat_lh.gii
b'created by chris on Sat Jun 20 22:04:11 2020\n'
saving to /home/chris/Dropbox/CURRENT_PROJECTS/NHP_MRI/pycortex/db/NMT/surfaces/flat_rh.gii
No overlays.svg file to delete


## [OPTIONAL] Create the Freesurfer based transforms
If the transforms from func/anat (reference) to the FS-anatomical on which the surfaces are based have not been created yet, do it here. If they do exist, skip these steps and move on to **Convert and import Freesurfer transforms into Pycortex**. We assume here that results are in functional space and need to be registered to the Freesurfer anatomical on which the surfaces are based. Adjust where necessary (in a new notebook). 

In [30]:
# where are functional references
epi = os.path.join(manmask_path, 'sub-danny', 'warps', 'sub-danny_func2nmt_res-1x1x1.nii.gz')
epi_mask = os.path.join(manmask_path, 'sub-danny', 'warps', 'sub-danny_func2nmt_mask_res-1x1x1.nii.gz')
epi_brain = os.path.join(manmask_path, 'sub-danny', 'warps', 'sub-danny_func2nmt_brain_res-1x1x1.nii.gz')

# where are the freesurfer anatomicals 
fs_t1 = os.path.join(fsp,'mri','brainmask.mgz')
fs_wm = os.path.join(fsp,'mri','wm.mgz')

# make a folder for reference-to-freesurfer files
mm2fs_path = os.path.join(fsp,'manual-masks_toFS') 
shutil.os.makedirs(mm2fs_path,exist_ok=True)

# convert mgz to nifti
subprocess.run(['mri_convert', fs_t1, os.path.join(fsp,'mri','brain.nii.gz')])
subprocess.run(['mri_convert', fs_wm, os.path.join(fsp,'mri','wm.nii.gz')])

CompletedProcess(args=['mri_convert', '/usr/local/freesurfer6/subjects/NMT/mri/wm.mgz', '/usr/local/freesurfer6/subjects/NMT/mri/wm.nii.gz'], returncode=0)

Now create the registration files.

In [31]:
# flirt
subprocess.run([
    'flirt', '-ref', os.path.join(fsp,'mri','brain.nii.gz'),
    '-wmseg', os.path.join(fsp,'mri','wm.nii.gz'),
    '-in', epi_brain,
    '-out', os.path.join(mm2fs_path,'epi2fs.nii.gz'),
    '-omat', os.path.join(mm2fs_path,'epi2fs.mat'),
    '-pedir','-2',
    ])

# tkreg
subprocess.run([
    'tkregister2', '--mov', epi_brain,
    '--targ', os.path.join(fsp,'mri','brain.nii.gz'),
    '--fsl',  os.path.join(mm2fs_path,'epi2fs.mat'),
    '--reg',  os.path.join(mm2fs_path,'epi2fs.reg.dat'),
    '--s', subject,
    ])

CompletedProcess(args=['tkregister2', '--mov', '/Users/chris/Dropbox/GIT_Support/NHP-BIDS/manual-masks/sub-danny/warps/sub-danny_func2nmt_brain_res-1x1x1.nii.gz', '--targ', '/usr/local/freesurfer6/subjects/NMT/mri/brain.nii.gz', '--fsl', '/usr/local/freesurfer6/subjects/NMT/manual-masks_toFS/epi2fs.mat', '--reg', '/usr/local/freesurfer6/subjects/NMT/manual-masks_toFS/epi2fs.reg.dat', '--s', 'NMT'], returncode=0)

## Convert and import freesurfer transforms into Pycortex
These transforms have probably already been generated in [NHP-Freesurfer](https://github.com/VisionandCognition/NHP-Freesurfer). 

In [32]:
# link to a folder for reference-to-freesurfer files
mm2fs_path = os.path.join(fsp,'manual-masks_toFS') 
# convert the existing FS transform from ref-functional to FS-anatomical into a pycortex format
xfm_name = "epi2surf" # how will we call this transform
# copy FS anatomy to expected filename (may not be necessary but is if you followed NHP-Freesurfer)
shutil.copyfile(os.path.join(fsp,'mri','T1.mgz'), os.path.join(fsp,'mri','orig.mgz'))
# set same shorter names for files
# the tkreg file from freesurfer
fs_dat_func = os.path.join(mm2fs_path,'epi2fs.reg.dat')
# convert
cortex.xfm.Transform.from_freesurfer(fs_dat_func,epi_brain,subject).save(subject,xfm_name,'coord')

In [33]:
# We'll create some random data here to test
data = cortex.Volume.random(subject, 'epi2surf')

In [34]:
# open web viewer
cortex.webshow(data)

Generating new ctm file...
wm
wm
inflated
inflated
Generating a flatmap cache
Started server on port 49601


<JS: window.viewer>

Stopping server


# Project results to the surface
The code below shows how to project results to the surfaces. 

In [ ]:
# You can load data with 
# test_data = <'path/to/testdata.nii.gz>'
# data = cortex.Volume(test_data, subject, xfm)

The loaded volume `test_data` is now a numpy array and you can manipulate it as such (use masks, threshold, interpolate, etc)

In [ ]:
# then look at the results
# cortex.webshow(data)